In [141]:
import os
import pandas as pd
import numpy as np
from progressbar import progressbar

In [188]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 400)
pd.options.mode.chained_assignment = None  # default='warn'

In [90]:
def process_box_scores(dir: str="../data/nba_boxscores", to_file=False) -> pd.DataFrame:
    def calculateFinalScore(df: pd.DataFrame)-> pd.DataFrame:
        '''Return gameid, teams playing, and final score
        
        Params:
            df: boxscore dataframe
        Returns:
            game_id, team1_id, team2_id, team1_score, team2_score
        '''
        game_id = df["GAME_ID"][0]
        team1_id = df["TEAM_ID"][0]
        team2_id = df["TEAM_ID"].iloc[-1]
        team1_score = df.loc[df["TEAM_ID"] == team1_id]["PTS"].sum()
        team2_score = df.loc[df["TEAM_ID"] == team2_id]["PTS"].sum()
        winner = 1 if team1_score > team2_score else 2
        boxscores_1 = df.iloc[:, 10:].loc[df["TEAM_ID"] == team1_id].sum().add_prefix("TEAM1_").to_frame().T
        boxscores_2 = df.iloc[:, 10:].loc[df["TEAM_ID"] == team2_id].sum().add_prefix("TEAM2_").to_frame().T

        result = pd.DataFrame([[game_id, team1_id, team2_id, team1_score, team2_score, winner]], columns=["GAME_ID", "TEAM1_ID", "TEAM2_ID", "TEAM1_SCORE", "TEAM2_SCORE", "WINNER"])
        result = pd.concat([result, boxscores_1, boxscores_2], axis=1)

        return result
    
    final_scores = []
    team_boxscores = []

    for filename in progressbar(os.listdir(dir)):
        f = os.path.join(dir, filename)
        # checking if it is a file
        if not os.path.isfile(f):
            continue
    
        df = pd.read_csv(f)
        bs = calculateFinalScore(df)

        team_boxscores.append(bs)
        final_scores.append(bs[["GAME_ID", "TEAM1_ID", "TEAM2_ID", "TEAM1_SCORE", "TEAM2_SCORE", "WINNER"]])
    
    final_scores_df = pd.concat(final_scores)
    team_boxscores_df = pd.concat(team_boxscores)


    if to_file:
        final_scores_df.to_csv("../data_proc/final_scores.csv", index=False)
        team_boxscores_df.to_csv("../data_proc/games_boxscores_master.csv", index=False)

    return final_scores_df

process_box_scores("../data/nba_boxscores", to_file=True)

100% (15365 of 15365) |##################| Elapsed Time: 0:00:41 Time:  0:00:41


,GAME_ID,TEAM1_ID,TEAM2_ID,TEAM1_SCORE,TEAM2_SCORE,WINNER
0,21600885,1610612737,1610612738,114.0,98.0,1
0,21900278,1610612741,1610612757,103.0,107.0,2
0,11200117,1610612756,1610612743,88.0,72.0,1
0,21400829,1610612738,1610612747,111.0,118.0,2
0,41900313,1610612747,1610612743,106.0,114.0,2
...,...,...,...,...,...,...
0,21800513,1610612752,1610612749,96.0,112.0,2
0,21700136,1610612737,1610612739,117.0,115.0,1
0,21400830,1610612755,1610612748,108.0,119.0,2
0,21600644,1610612758,1610612763,91.0,107.0,2


In [92]:
from helpers.csv_helpers import combine_csv_dir

def game_id_df(dir: str="../data/nba_gamelogs/", update_master=False, to_file=False) -> pd.DataFrame:
    '''Mapping df between NBA Game_ID (ex. 21600885) and betting (ex. "20231111_nba_Miami_Atlanta") IDs 
    
    Params:
        dir: directory of nba gamelogs
        join_files: create/use processed combined gamelogs
        reload_master: update processed combined gamelogs
        to_file: write to file
    '''
    lu = pd.read_csv("../data/lu/nba_team_lu.csv")
    lu.drop(["NBA_team_name", "team_id"], axis=1, inplace=True)
    lu.dropna(inplace=True)
    abbrv_to_sbr = lu.set_index("NBA_team_abbrev").T.to_dict('records')[0]

        
    df = combine_csv_dir(dir)

    df.drop_duplicates(["TEAM_ID", "GAME_ID"], inplace=True)
    game_id_df = df[["GAME_ID", "GAME_DATE", "MATCHUP"]]
    game_id_df["GAME_DATE"] = game_id_df["GAME_DATE"].apply(lambda x: x.split("T")[0].replace("-", ""))
    game_id_df[["TEAM1", "TEAM2"]] = game_id_df["MATCHUP"].str.split(r' .* ', expand=True)
    game_id_df["TEAM1"] = game_id_df["TEAM1"].map(abbrv_to_sbr)
    game_id_df["TEAM2"] = game_id_df["TEAM2"].map(abbrv_to_sbr)

    game_id_df["SBR_GAME_ID"] = game_id_df["GAME_DATE"] + "_nba_" + game_id_df["TEAM1"] + "_" + game_id_df["TEAM2"]
    game_id_df = game_id_df[["GAME_ID", "SBR_GAME_ID"]]

    if to_file:
        game_id_df.to_csv("game_id_proc.csv", index=False)
    return game_id_df


game_id_df("../data/nba_gamelogs/", to_file=True, update_master=True)

100% (12 of 12) |########################| Elapsed Time: 0:00:01 Time:  0:00:01


,GAME_ID,SBR_GAME_ID
0,11800001,NaN
9,11800001,NaN
26,11800002,20180928_nba_Charlotte_Boston
28,11800002,20180928_nba_Boston_Charlotte
53,11800003,20180929_nba_Toronto_Portland
...,...,...
30539,41400404,20150611_nba_Golden State_Cleveland
30558,41400405,20150614_nba_Cleveland_Golden State
30560,41400405,20150614_nba_Golden State_Cleveland
30575,41400406,20150616_nba_Cleveland_Golden State


In [15]:
def games_basic_master(boxscore_dir:str="../data/nba_boxscores", 
                        gamelogs_dir:str="../data/nba_gamelogs/", 
                        bets_dir:str="../data/nba_lines_historical",
                        to_file=False) -> pd.DataFrame:
    '''Generates master games dataframe with game id's, final scores, and betting lines

    Params:
        boxscore_dir: directory for boxscores, runs process_box_scores on directory if no csv detected
        gamelogs_dir: directory for gamelogs, runs game_id_df on directory if no csv detected
        bets_dir: directory for betting lines, combines with boxscore and gamelogs
    '''

    if not os.path.isfile("final_scores.csv"):
        finalscore_df = process_box_scores(boxscore_dir)
    else:
        finalscore_df = pd.read_csv("final_scores.csv")

    if not os.path.isfile("game_id_proc.csv"):
        game_id_df = game_id_df(gamelogs_dir)
    else:
        game_id_df = pd.read_csv("game_id_proc.csv")
    
    master_df = pd.DataFrame()

    # process bets

    # moneylines -> probabilities
    
    f = os.path.join(bets_dir, f"nba_historical_moneyline.csv")

    df = pd.read_csv(f)
    df.dropna(inplace=True)
    df["away_prob"] = df["away_line"].apply(lambda x: -100 / x if x < 0 else x / 100)
    df["away_prob"] = 1 - df["away_prob"] / (df["away_prob"] + 1)
    df["home_prob"] = df["home_line"].apply(lambda x: -100 / x if x < 0 else x / 100)
    df["home_prob"] = 1 - df["home_prob"] / (df["home_prob"] + 1)
    df.drop(df[(df["away_prob"] + df["home_prob"] >= 1.4)].index, inplace=True) # 1.3 is profit margin for sports bettings
    df.drop(["sport", "bet_type", "away", "home", "home_line", "away_line", "home_prob"], axis=1, inplace=True)

    bets_master = df

    # spread
    f = os.path.join(bets_dir, f"nba_historical_spread.csv")

    df = pd.read_csv(f)
    df.dropna(inplace=True)
    df.drop(df[(df["away_line"] != -df["home_line"])].index, inplace=True) # home line opposite of away line
    df.drop(["sport", "date", "bet_type", "away", "home", "home_line"], axis=1, inplace=True)
    df.rename(columns={"away_line": "away_spread"}, inplace=True)
    bets_master = bets_master.merge(df, on="game_id", how="outer")

    # total
    f = os.path.join(bets_dir, f"nba_historical_total.csv")

    df = pd.read_csv(f)
    df.dropna(inplace=True)
    df.drop(df[(df["away_line"] != df["home_line"])].index, inplace=True) # home line equal away line
    df.drop(["sport", "date", "bet_type", "away", "home", "home_line"], axis=1, inplace=True)
    df.rename(columns={"away_line": "away_total"}, inplace=True)
    bets_master = bets_master.merge(df, on="game_id", how="outer")

    bets_master.dropna(subset=["away_prob", "away_spread", "away_total"], inplace=True)
    
    # final scores
    master_df = finalscore_df

    # game ids
    master_df = master_df.merge(game_id_df, on="GAME_ID")
    master_df.insert(1, "SBR_GAME_ID", master_df.pop("SBR_GAME_ID"))
    
    # betting lines
    master_df = master_df.merge(bets_master, left_on="SBR_GAME_ID", right_on="game_id")
    master_df.drop("game_id", axis=1, inplace=True)
    master_df.insert(2, "DATE", master_df.pop("date"))

    if to_file:
        master_df.to_csv("games_basic_master.csv", index=False)

    return master_df
    

games_basic_master(to_file=True)

,GAME_ID,SBR_GAME_ID,DATE,TEAM1_ID,TEAM2_ID,TEAM1_SCORE,TEAM2_SCORE,WINNER,away_prob,away_spread,away_total
0,21600885,20170227_nba_Atlanta_Boston,20170227.0,1610612737,1610612738,114.0,98.0,1,0.392157,4.5,212.5
1,21900278,20191129_nba_Chicago_Portland,20191129.0,1610612741,1610612757,103.0,107.0,2,0.277778,7.0,227.0
2,41900313,20200922_nba_L.A. Lakers_Denver,20200922.0,1610612747,1610612743,106.0,114.0,2,0.700599,-6.0,211.5
3,22100624,20220112_nba_Cleveland_Utah,20220112.0,1610612739,1610612762,111.0,91.0,1,0.363636,5.5,222.5
4,22000156,20210111_nba_Indiana_Sacramento,20210111.0,1610612754,1610612758,122.0,127.0,2,0.622642,-3.5,229.0
...,...,...,...,...,...,...,...,...,...,...,...
9645,21800513,20181227_nba_New York_Milwaukee,20181227.0,1610612752,1610612749,96.0,112.0,2,0.106724,14.0,226.5
9646,21700136,20171105_nba_Atlanta_Cleveland,20171105.0,1610612737,1610612739,117.0,115.0,1,0.140056,11.0,220.0
9647,21400830,20150223_nba_Philadelphia_Miami,20150223.0,1610612755,1610612748,108.0,119.0,2,0.148148,11.5,193.5
9648,21600644,20170120_nba_Sacramento_Memphis,20170120.0,1610612758,1610612763,91.0,107.0,2,0.256410,8.0,199.5


In [194]:
def game_wl_records(dir:str=None, to_file=False) -> pd.DataFrame:
    '''Generate dataframe of games with previous win/loss of each games
    
    Params:
        dir: directory of basic master, generates if none
    '''
    if dir is None:
        dir = "games_basic_master.csv"
    
    lu = pd.read_csv("../data/lu/nba_team_lu.csv")
    lu.drop(["NBA_team_name"], axis=1, inplace=True)
    lu.dropna(inplace=True)

    df = pd.read_csv(dir)
    # df["date_m"] = pd.to_datetime(df["DATE"], format="%Y%m%d")
    df[["TEAM1_10wp", "TEAM1_25wp", "TEAM1_50wp", "TEAM1_75wp", "TEAM1_100wp", "TEAM2_10wp", "TEAM2_25wp", "TEAM2_50wp", "TEAM2_75wp", "TEAM2_100wp"]] = np.nan

    for team_id in progressbar(lu["team_id"].unique()):
        # team_id = lu["team_id"].unique()[0]
        team_df = df.loc[(df["TEAM1_ID"] == team_id) | (df["TEAM2_ID"] == team_id)].sort_values(by="DATE")

        team_df["won_game"] = -1 * (team_df["WINNER"] - 2)
        team_df.loc[df["TEAM2_ID"] == team_id, "won_game"]= -1 * team_df["won_game"] + 1

        team_df["10wp"] = team_df["won_game"].rolling(10, min_periods=10).mean().shift(1)
        team_df["25wp"] = team_df["won_game"].rolling(25, min_periods=25).mean().shift(1)
        team_df["50wp"] = team_df["won_game"].rolling(50, min_periods=50).mean().shift(1)
        team_df["75wp"] = team_df["won_game"].rolling(75, min_periods=75).mean().shift(1)
        team_df["100wp"] = team_df["won_game"].rolling(100, min_periods=100).mean().shift(1)
        team_df = team_df[["GAME_ID", "10wp", "25wp", "50wp", "75wp", "100wp"]]
        team_df.insert(1, "ID", team_id)

        df.update(team_df.rename(columns={c: "TEAM1_" + c for c in team_df.columns if c not in ['GAME_ID']}))
        # display(df)
        df.update(team_df.rename(columns={c: "TEAM2_" + c for c in team_df.columns if c not in ['GAME_ID']}))
        # display(df)

    df.sort_values(by="DATE", inplace=True)

    if to_file:
        df.to_csv("games_wp_master.csv")

    return df

game_wl_records(to_file=True)


100% (30 of 30) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


,GAME_ID,SBR_GAME_ID,DATE,TEAM1_ID,TEAM2_ID,TEAM1_SCORE,TEAM2_SCORE,WINNER,away_prob,away_spread,away_total,TEAM1_10wp,TEAM1_25wp,TEAM1_50wp,TEAM1_75wp,TEAM1_100wp,TEAM2_10wp,TEAM2_25wp,TEAM2_50wp,TEAM2_75wp,TEAM2_100wp
7493,21400001,20141028_nba_Orlando_New Orleans,20141028.0,1610612740,1610612740,84.0,101.0,2,0.208333,9.00,191.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8156,21400003,20141028_nba_Houston_L.A. Lakers,20141028.0,1610612745,1610612745,108.0,90.0,1,0.750000,-7.00,207.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8108,21400002,20141028_nba_Dallas_San Antonio,20141028.0,1610612742,1610612742,100.0,101.0,2,0.416667,3.50,203.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9348,21400010,20141029_nba_Chicago_New York,20141029.0,1610612741,1610612741,104.0,80.0,1,0.661017,-4.50,184.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9355,21400011,20141029_nba_Detroit_Denver,20141029.0,1610612743,1610612743,79.0,89.0,2,0.307692,6.50,205.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3393,22300011,20231110_nba_New Orleans_Houston,20231110.0,1610612745,1610612745,101.0,104.0,2,0.439560,3.00,221.00,0.4,0.32,0.30,0.280000,0.25,0.4,0.32,0.30,0.280000,0.25
2795,22300013,20231110_nba_Minnesota_San Antonio,20231110.0,1610612750,1610612750,117.0,110.0,1,0.749355,-7.25,225.00,0.7,0.60,0.50,0.506667,0.50,0.7,0.60,0.50,0.506667,0.50
2789,22300012,20231110_nba_Utah_Memphis,20231110.0,1610612763,1610612763,127.0,121.0,1,0.378072,5.00,230.25,0.3,0.44,0.56,0.626667,0.60,0.3,0.44,0.56,0.626667,0.60
4037,22300015,20231110_nba_L.A. Lakers_Phoenix,20231110.0,1610612747,1610612747,122.0,119.0,1,0.481928,2.00,224.75,0.5,0.72,0.62,0.493333,0.48,0.5,0.72,0.62,0.493333,0.48
